In [1]:
import cv2, time, os, tensorflow as tf
import numpy as np
from tensorflow.python.keras.utils.data_utils import get_file

In [2]:
np.random.seed(12)

In [3]:
classesFilePath = './coco.names'

# modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz"
modelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz"
imagePath = "../../Data/helmet/images/BikesHelmets56.png"

In [4]:
with open(classesFilePath, "r") as f:
    classesList = f.read().splitlines()

colorList = np.random.uniform(low=0, high=255, size=(len(classesList), 3))
print(len(classesList), len(colorList))

92 92


### Downloading TFOD Model

In [5]:
fileName = os.path.basename(modelURL)
modelName = fileName[:fileName.index('.')]
print("fileName: ", fileName, "ModelName: ", modelName)

cacheDir = "./pretrained_models"
# Creating the models directory if not existed before
os.makedirs(cacheDir, exist_ok=True)
get_file(fileName, origin=modelURL, cache_dir=cacheDir, cache_subdir="models", extract=True)

fileName:  efficientdet_d1_coco17_tpu-32.tar.gz ModelName:  efficientdet_d1_coco17_tpu-32


'./pretrained_models\\models\\efficientdet_d1_coco17_tpu-32.tar.gz'

In [6]:
tf.keras.backend.clear_session()
model = tf.saved_model.load(os.path.join(cacheDir, "models", modelName, "saved_model"))

In [52]:
# Loading and converting an image to tensor for test purpose
image = cv2.imread(imagePath)
imageRGB = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2RGB)
imageTensor = tf.convert_to_tensor(imageRGB, dtype=tf.uint8)
imageTensor = imageTensor[tf.newaxis, ...]

In [53]:
detections = model(imageTensor)

In [60]:
def createBoundingBox(image, detections, threshold = 0.5):
    bboxes = detections['detection_boxes'][0].numpy()
    classIndexes = detections['detection_classes'][0].numpy().astype(np.int32)
    classScores = detections['detection_scores'][0].numpy()

    imH, imW, imC = image.shape
    image = cv2.resize(image, (1000, int(imH * (1000/imW))))
    imH, imW = (int(imH * (1000/imW)), 1000)

    # gives indexes of bboxes with the criteria
    bboxIds = tf.image.non_max_suppression(bboxes, classScores, max_output_size=50, iou_threshold=threshold, score_threshold=threshold)


    if len(bboxes):
        for i in bboxIds:
            bbox = tuple(bboxes[i])
            classIndex = classIndexes[i]
            classLabel = classesList[classIndex]
            classConfidence = round(100*classScores[i])
            classColor = colorList[classIndex]

            displayTxt = f'{classLabel}: {classConfidence}%'
            # displayTxt = '{}: {}%'.format(classLabel, classConfidence)
            ymin, xmin, ymax, xmax = bbox   # in relative format
            ymin, xmin, ymax, xmax = (int(ymin*imH), int(xmin*imW), int(ymax*imH), int(xmax*imW))

            cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color=classColor, thickness=1)
            cv2.putText(image, displayTxt, (xmin, ymin - 7), cv2.FONT_HERSHEY_COMPLEX,  1, classColor, 2)

    cv2.imwrite("./detections/" + modelName + ".jpg", image)
    cv2.imshow("Object detection", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [61]:
createBoundingBox(image, detections)

In [ ]:
vdo = cv2.VideoCapture("data/v2.mp4")

t1 = 0
while True:
    captured, frame = vdo.read()

    frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    if result.detections:
        for ids, detection in enumerate(result.detections):
            faceDraw.draw_detection(frame, detection)
            # print(ids, detection)
            print(len(result.detections))


    t2 = time.time()
    fps = 1/(t2 - t1)
    t1 = t2

    cv2.putText(frame, f"FPS: {int(fps)}", (30, 30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 200, 0), 1)
    cv2.imshow("Video", frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cv2.waitKey(0)
vdo.release()
cv2.destroyAllWindows()